# F0 and F1 Estimation

## F0 Estimate via Ideal F0 Estimation
The ideal F0 Estimation algorithm estimates the number of distinct elements in a data stream/set (F0)


## Material to Review
Please review the class slides for a refresher on the algorithm, linked <a href="https://docs.google.com/presentation/d/13aaPw-0sT6OYvrdn0JlR9dGOJ2tZE_DMzUu2MZrMnJY/edit?usp=sharing"> here (SLIDE 7) </a> Two algorithms for FM algorithm are given in the slides, please implement the "idealized f0 algorithm"

For review of k-independent hash family construction, please refer to this <a href="https://drive.google.com/file/d/1A0-QxDYag8ODiVfFCrAm0OINZv-seKw0/view"> link (SLIDE 7) </a>

## Data
We simulate a data stream by generating a numpy array via the function "generateSeq()". In your algorithm you should treat this array as if it were a data stream.

## Problem statement

Please implement the idealized F0 estimation algorithm as described in the review material. 

In order to implement the algorithm, you will need a random has function h:U -> [0,1]. Notice that this hash function takes a number and hashes it to a value from range 0 to 1. 

We ask you to run experiments on the FM algorithm with 3 different types of hashes:
1. 2-wise independent hash family
2. 4-wise independent hash family
3. murmuhash.hash(x, signed = False) - a function from a popular, publicly available hash library, murmurhash

For each type of hash, we ask you to run 1,000 trials where each trial will produce its own estimate of F0. Afterwards, you will average the 1,000 estimates of F0 to produce a final estimate. 

### 2-wise independent hash family and 4-wise independent hash family

Please refer to the Material to Review section for information on how to construct a k-wise independent hash family.

For your prime value, p, we have provided the variable "LARGE_PRIME", which you can use for all hashes. 

Please complete the following 4 functions: 

1. generateHashCoefficients(k, p) - generates a list of length k which represents the polynomial coefficients of the hash
2. khash(coeffs, p, val) - hashes val given the polynomial coefficients, large prime number p, and the value, val. 
3. kWiseHashFMAlg - ideal F0 estimation using k-wise independent hashing
4. murmurHashFMAlg - ideal F0 estimation using murmurhash

Please find a description for each of the functions underneath the function header.

Please use the variable "LARGE_PRIME" as your value for your prime number, p. Remember that the ultimate output of your hash functions is a value between 0 and 1, so the last step of the function "khash(coeffs, p, val)" would be to divide by p. 

### murmurhash

Murmurhash is a popular general purpose hash function. The documentation for this library can be found <a href="https://pypi.org/project/mmh3/"> here </a>. Installation instructions can be found in the URL provided as well.

For our purposes, the only function we will be concerned with is "murmurhash.hash(x, signed = False)" which takes as input, x, and outputs a 32-bit unsigned integer that is the mmurmur hash of x. 

**Note:** In the previous hash function, we divided by p at the end, however in the case of murmurhash, in order to get a final value between 0 and 1, we want to divide by the maximum output of "murmurhash.hash(x)". Since this function returns a 32 bit unsigned int, the maximum value is therefore $2^{32}$

**Submitted solutions that do not implement the idealized F0 estimation algorithm and do not use the implementation will receive a score of 0 even if you report the correct F0 value**

## Useful libraries (optional)
A useful progressbar library you can use is called "tqdm", to install simply pip install or conda install "tqdm" on the command line.

The progress bar creates a visual tracker for code loops and even estimates the exected amount of time the loop will take to finish. This is useful for this assignment in particular as repeating the estimation several thousand times may take several seconds and the progress bar provides evidence that your code has not entered an infinite loop.

To add a progress bar for a for loop, simply adjust the for loop from: 
"for i in range(x): " -> "for i in tqdm(range(x)): "

More information on tqdm API can be found <a herf="https://github.com/tqdm/tqdm"> here </a>

**Important note:** When importing the tqdm library, you will need to use the following import command "from tqdm import tqdm". If you call "import tqdm" you will likely run into an error when you try to use the tqdm progress bar. 

In [20]:
import numpy as np
import math
import random
random.seed(None)
from tqdm import tqdm
import mmh3

LARGE_PRIME = 157
n_trials = 1000

def generateSeq():
    arr=np.random.randint(0,100, size=(5000))
    return arr
    
def generateHashCoefficients(k,p):
    '''
    Input:
        k: int, represents the k-wise independent hash family
        p: int, large prime number given in "FINAL_LARGE_PRIME"
    Output:
        List of length k which represents the coefficients of k-length polynomial from x^0 to x^(k-1)
        
    generates the coefficients of a k-wise independent hash family represented by a polynomial function
    
    Each coefficient is drawn UAR from [p]
    Examples:
        24x^2 + 8x + 4 is represented as the list [4, 8, 24]
        77x + 12 is repsented as the list [12, 77]

    '''
    return np.random.choice(range(1,p), k)


def kWiseHash(coeffs, p, x):
    '''
    Input:
        coeffs: List of length k, which is generated by "generateHashCoefficients" function
        p: int, large prime number given in "FINAL_LARGE_PRIME"
        x: value to be hashed
        
    Output:
        value between 0 and 1 which is the hash(x)/p
    
    hashes the value, x, based on the polynomial coefficients generated from "generatehashCoefficients", and the prime number p
    '''
    sum = 0
    for i in range(len(coeffs)):
        sum += coeffs[i] * (x**(len(coeffs) - (i+1)))
    sum = sum % p
    sum = sum/p
    return sum

def kWiseHashFMAlg(k):
    '''
    Input:
        k = int, represents k-wise independent hash family. For this problem k = 2 and k = 4
        
    Output:
        None

    Print out the average F0 estimation accross 1000 trials for k-wise independent hash family

    Note, for each trial, the polynomial coefficients should be regenerated using "generateHashCoefficients".
    But within a single trial, the same hash function should be applied to all data points
    '''
    trials = []
    inpData=(generateSeq())
    for i in tqdm(range(n_trials)):

        V = float('inf')
        while True:
            V = float('inf')
            hashCoeffs = generateHashCoefficients(k, LARGE_PRIME)
            # print(hashCoeffs)
            for x in inpData:
                hash = kWiseHash(hashCoeffs, LARGE_PRIME, x)
            
                if hash < V:
                    V = hash

            if V != 0:
                break


        # if V != 0:
        trials.append((1/V)-1)
    print(np.mean(trials))

def murmurHashFMAlg():
    '''
    Input: None
    Output: None
    
    Print out the average F0 estimation accross 1000 trials for ideal F0 estimation using murmurHash
    '''
    
    trials = []
    inpData=(generateSeq())
    for i in tqdm(range(n_trials)):
        V = float('inf')
        
        while True:
            V = float('inf')
            # hashCoeffs = generateHashCoefficients(k, LARGE_PRIME)
            # print(hashCoeffs)
            for x in inpData:
                hash = mmh3.hash(x, signed = False)/(2**32)
            
                if hash < V:
                    V = hash

            if V != 0:
                break


        # if V != 0:
        trials.append((1/V)-1)
    print(np.mean(trials))




#your code for testing below

In [21]:
kWiseHashFMAlg(2)
kWiseHashFMAlg(4)
murmurHashFMAlg()


100%|██████████| 1000/1000 [00:05<00:00, 179.14it/s]

105.38542825092874


## F1 Estimate via Morris Algorithm
The Morris algorithm estimates the number of elements in a data stream (F1).


## Material to Review
Please review the Morris algorithm from the classs slides linked <a href="https://docs.google.com/presentation/d/14vmM5xsKnVBFMIHZLahYuQzkuOwlXsRwQXzRq0DqT1w/edit#slide=id.g117d4edbd70_1_141"> here </a>

## Data
The same generateSeq() function from F0 estimation is copied below, and will be used in this problem as well

## Problem Statement
Please implement the Morris Algorithm as described in the Material to Review

Similarily for the F1 estimate, the estimation can differ from trial to trial, therefore we ask that you repeat the MorrisCount algorithm 1,000 times recording the estimation of each trial and then average all of the trials together for the final estimation. 

**Submitted solutions that do not implement the Morris algorithm and do not use the implementation will receive a score of 0 even if they arrive at the correct F1 value**

In [31]:
import numpy as np
import math
from tqdm import tqdm

def generateSeq():
    arr=np.random.randint(0,9999, size=(10000))
    return arr

#your code for morris algorithm below

def morris_counter():
        
    trials = []

    inpData=(generateSeq())
    for i in tqdm(range(100)):
        c = 0
        for item in inpData:
            if np.random.choice([0,1], p = [1 - 1/(2**c), 1/(2**c)]):
                c += 1
        trials.append(2**c - 1)

    print(np.mean(trials))

morris_counter()


100%|██████████| 100/100 [00:49<00:00,  2.02it/s]

9993.24
